In [1]:
from __future__ import print_function
import sys
import os
import logging
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics
%config Completer.use_jedi = False
import SimpleITK as sitk
import nibabel as nib
import glob
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import logging
from utils_mask_sain import *
from utils import *
from utils_recaler import *
from utils_autour_3D import *
from area_calculus import *

In [2]:
path_data_brut = "../data/radio_brut"
current_dir = os.getcwd()
if current_dir == "/gpfs/users/selvestra/python_codes":
    path_data_brut = "/gpfs/workdir/selvestra/data/radio_brut"
path_radios = './*_NAT*/*.nii'
path_brut = os.path.join(path_data_brut, path_radios)
path_data = os.path.dirname(path_data_brut)
ls_image = sorted(glob.glob(path_brut))


time_inj = ["_ART", "_PORT", "_TARD"]
ls_image_prime = ls_image
ls_image_no_time = natsorted(
    list(set([eliminate_temps(x) for x in ls_image_prime])))
ls_image_full_times_surconfiance = [
    [name + time + ".nii" for time in time_inj] for name in ls_image_no_time]
ls_image_full_time = []
dict_image_full_time = {}
for li_names in ls_image_full_times_surconfiance:
    li_true_names = []
    for i, name in enumerate(li_names):
        if name in ls_image_prime:
            li_true_names.append(name)
    ls_image_full_time.append(li_true_names)
    classe_name = li_names[0].split('/')[-2].split('_')[1]
    patient_num = li_names[0].split('/')[-1].split('_')[0]
    dict_image_full_time[(patient_num, classe_name)] = li_true_names

ls_image_full_time = ls_image_full_time
    
median_spacing = [0.59375, 0.59375, 1.5]

# Visualiser à une depth donnée

In [3]:
def visualize_depth(li_images, li_masks, li_depths, resolution = 0.3):
    for depth in li_depths:
        plt.figure(figsize = (15,5))
        for i, image in enumerate(li_images):
            mask = li_masks[i]
            good_spacing = median_spacing[:2]
            good_spacing.append(resolution)
            new_image = resample_image_to_reference(image, image, force_spacing= good_spacing,  force_size = None, interpolator = "bspline")
            new_mask = resample_image_to_reference(mask, mask, force_spacing = good_spacing, interpolator = "nearest")
            origin = new_image.GetOrigin()
            slice_num = new_image.TransformPhysicalPointToContinuousIndex([origin[0],origin[1],depth])[2]
            slice_image  = new_image[:,:,int(slice_num)]
            slice_mask = new_mask[:,:,int(slice_num)]
            mask_superpose_simple(slice_image, slice_mask, other_mask = None,num = i + 1, max_num = len(li_images), legend = f"Image  at time {i + 1} at depth {depth} mm")
        plt.show()

In [4]:
# num = 30
# li_images = [sitk.ReadImage(image_name) for image_name in ls_image_full_time[num]]
# li_masks = [sitk.ReadImage(image_name.replace('_NAT','').replace('.nii','_masked.nii')) != 0 for image_name in ls_image_full_time[num]]
# visualize_depth(li_images, li_masks, [69.5, 69.5 + 1.6/2 - 0.3, 69.5+1.6/2 +0.3 ])

# Par slice

In [5]:
def visualize_slice(li_images, li_masks,deep_li_slices,li_scales = None):
    if li_scales is None:
        new_li_images = [image for image in li_images]
        new_li_masks = [mask for mask in li_masks]
    else:
        new_li_masks = []
        new_li_images = []
        for i, image in enumerate(li_images):
            mask = li_masks[i]
            print(image.GetSpacing(), image.GetSize())
            new_image  = resample_image_to_reference(image, image, force_spacing= li_scales[i],  force_size = None, interpolator = "bspline")
            new_mask = resample_image_to_reference(mask, mask, force_spacing = li_scales[i], interpolator = "nearest")
            new_li_images.append(new_image)
            new_li_masks.append(new_mask)
    if deep_li_slices is None:
        #indispensable d'avoir la même échelle aux trois images dans ce cas
        deep_li_slices = [list(range(new_li_images[0].GetSize()[2])) for image in new_li_images]
    for slice_ref in range(len(deep_li_slices[0])):
        plt.figure(figsize = (15,5))
        for image_num, image in enumerate(new_li_images):
            mask = new_li_masks[image_num]
            num_slice = deep_li_slices[image_num][slice_ref]
            slice_image = image[:,:,num_slice]
            slice_mask = mask[:,:,num_slice]
            mask_superpose_simple(slice_image, slice_mask, other_mask = None,num = image_num + 1, max_num = len(li_images), legend = f"Image  at time {image_num + 1} at slice {num_slice} \n depth = {image.TransformIndexToPhysicalPoint([0,0,num_slice])[2]} mm,\n spacing = {image.GetSpacing()[2]} mm")
        plt.show()
        
    

In [6]:
# num = 5
# print(ls_image_full_time[num])
# li_images = [sitk.ReadImage(image_name) for image_name in ls_image_full_time[num]]
# li_masks = [sitk.ReadImage(image_name.replace('_NAT','').replace('.nii','_masked.nii')) != 0 for image_name in ls_image_full_time[num]]
# deep_li_slices = [[2303, 2318, 2335, 2351, 2366, 2382, 2398, 2411, 2421, 2432], [2308, 2323, 2340, 2355, 2370, 2386, 2401, 2415, 2425, 2435], [1247, 1261, 1271, 1281, 1291, 1302, 1315, 1327, 1338, 1351]]
# #li_scales = [np.copy(median_spacing), np.copy(median_spacing), np.copy(median_spacing)]
# li_scales = [np.copy(li_images[i].GetSpacing()) for i in range(len(li_images))]
# # li_scales[0][2] = 0.04057762983078987
# # li_scales[1][2] = 0.040568852630042326
# # li_scales[2][2] = 0.07403103169022245
# visualize_slice(li_images, li_masks,deep_li_slices = None,li_scales = None)

